# 🎮 Sopa de Letras Semántica con Word2Vec

**Diplomatura en Inteligencia Artificial - Universidad de Palermo**

Juego interactivo que demuestra la capacidad de Word2Vec para capturar relaciones semánticas.

---

## 📌 Objetivo Pedagógico

Este juego visualiza cómo Word2Vec captura **relaciones semánticas** entre palabras:

- Las palabras ocultas están **ordenadas por similitud coseno** con la palabra objetivo
- Los **puntos** reflejan el score de similitud (0-100)
- Demuestra que el modelo aprendió contextos correctos durante el entrenamiento

**Concepto clave**: Si encuentras "LOVE" cuando la palabra objetivo es "HAPPY", ganas 85 puntos porque `similarity('happy', 'love') = 0.85`


In [1]:
from gensim.models import Word2Vec
from pathlib import Path
from IPython.display import display, HTML
import random
import json
import numpy as np

In [2]:
# Cargar modelo Word2Vec entrenado
BASE_DIR = Path('..').resolve()
model_path = BASE_DIR / 'models' / 'word2vec_model.pkl'

print("🔄 Cargando modelo Word2Vec...")
model_w2v = Word2Vec.load(str(model_path))
print(f"✅ Modelo cargado: {len(model_w2v.wv):,} palabras en vocabulario\n")

🔄 Cargando modelo Word2Vec...
✅ Modelo cargado: 57,795 palabras en vocabulario



## 🧠 CALIBRACIÓN DEL MOTOR SEMÁNTICO

Antes de jugar, vamos a **demostrar** que este NO es un juego con palabras pre-programadas.

El motor usa **Word2Vec** para:
1. **`most_similar()`** - Encontrar palabras cercanas en el espacio vectorial
2. **Similitud Coseno** - Calcular qué tan relacionadas están dos palabras
3. **Analogías Semánticas** - Resolver "rey - hombre + mujer = reina"

👇 **Ejecuta la siguiente celda para ver el motor en acción:**

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🧠 DEMOSTRACIÓN: MOTOR SEMÁNTICO WORD2VEC EN ACCIÓN
# ═══════════════════════════════════════════════════════════════════════════════
# Esto demuestra que el juego NO usa listas pre-programadas.
# El modelo genera palabras dinámicamente basándose en SEMÁNTICA VECTORIAL.

from IPython.display import display, HTML
import time

def demo_motor_semantico():
    """Demostración visual de las capacidades de Word2Vec"""
    
    print("=" * 70)
    print("🔬 CALIBRANDO MOTOR SEMÁNTICO WORD2VEC...")
    print("=" * 70)
    time.sleep(0.5)
    
    # ═══════════════════════════════════════════════════════════════════════════
    # DEMO 1: most_similar() - Búsqueda de palabras cercanas
    # ═══════════════════════════════════════════════════════════════════════════
    print("\n📊 TEST 1: most_similar() - Palabras cercanas en espacio vectorial")
    print("-" * 70)
    
    palabras_test = ['happy', 'sad', 'love']
    for palabra in palabras_test:
        similares = model_w2v.wv.most_similar(palabra, topn=5)
        print(f"\n   🎯 '{palabra.upper()}' → Top 5 más similares:")
        for i, (p, sim) in enumerate(similares, 1):
            barra = "█" * int(sim * 20)
            print(f"      {i}. {p:<12} │{barra:<20}│ {sim:.4f}")
    
    # ═══════════════════════════════════════════════════════════════════════════
    # DEMO 2: Similitud Coseno entre pares de palabras
    # ═══════════════════════════════════════════════════════════════════════════
    print("\n\n📐 TEST 2: Similitud Coseno - Distancia semántica")
    print("-" * 70)
    
    pares = [
        ('happy', 'joy'),
        ('happy', 'sad'),
        ('love', 'hate'),
        ('good', 'great'),
        ('work', 'job')
    ]
    
    for p1, p2 in pares:
        try:
            sim = model_w2v.wv.similarity(p1, p2)
            barra = "█" * int(abs(sim) * 30)
            emoji = "✅" if sim > 0.5 else "⚠️" if sim > 0 else "❌"
            print(f"   {emoji} '{p1}' ↔ '{p2}': {sim:.4f} │{barra}│")
        except:
            print(f"   ❌ Par no encontrado: {p1} - {p2}")
    
    # ═══════════════════════════════════════════════════════════════════════════
    # DEMO 3: Analogías Semánticas (el "kill shot")
    # ═══════════════════════════════════════════════════════════════════════════
    print("\n\n🧮 TEST 3: ANALOGÍAS SEMÁNTICAS (A es a B como C es a ???)")
    print("-" * 70)
    print("   Esto demuestra que Word2Vec captura RELACIONES, no solo palabras.\n")
    
    analogias = [
        # (positivas, negativas, explicación)
        (['good', 'bad'], ['happy'], "Si 'good' es opuesto de 'bad', entonces 'happy' es opuesto de..."),
        (['love', 'loved'], ['hate'], "Si 'love' → 'loved', entonces 'hate' →..."),
        (['day', 'night'], ['morning'], "Si 'day' es opuesto de 'night', 'morning' es opuesto de..."),
    ]
    
    for positivas, negativas, explicacion in analogias:
        try:
            resultado = model_w2v.wv.most_similar(positive=positivas, negative=negativas, topn=3)
            print(f"   💡 {explicacion}")
            print(f"      Ecuación: {' + '.join(positivas)} - {' - '.join(negativas)} = ?")
            print(f"      Resultado del modelo:")
            for i, (palabra, score) in enumerate(resultado[:3], 1):
                print(f"         {i}. {palabra.upper()} (score: {score:.4f})")
            print()
        except Exception as e:
            print(f"   ⚠️ Analogía no disponible: {e}\n")
    
    # ═══════════════════════════════════════════════════════════════════════════
    # DEMO 4: Generación Dinámica (prueba que NO hay lista fija)
    # ═══════════════════════════════════════════════════════════════════════════
    print("\n🎲 TEST 4: GENERACIÓN DINÁMICA DE NIVELES")
    print("-" * 70)
    print("   Prueba de que el juego puede generar niveles para CUALQUIER palabra:\n")
    
    palabras_random = ['twitter', 'music', 'food', 'money', 'sleep']
    for palabra in palabras_random:
        if palabra in model_w2v.wv:
            similares = model_w2v.wv.most_similar(palabra, topn=3)
            palabras_nivel = [p[0].upper() for p in similares]
            print(f"   🎮 Nivel '{palabra.upper()}': {' → '.join(palabras_nivel)}")
        else:
            print(f"   ❌ '{palabra}' no está en vocabulario")
    
    print("\n" + "=" * 70)
    print("✅ MOTOR SEMÁNTICO CALIBRADO - LISTO PARA JUGAR")
    print("=" * 70)
    print("\n💡 CONCLUSIÓN: Este juego usa GENERACIÓN PROCEDURAL basada en")
    print("   semántica vectorial. NO hay niveles pre-programados.")
    print("   Cada palabra genera un nivel único usando most_similar().\n")

# Ejecutar demostración
demo_motor_semantico()

In [3]:
# Diccionario bilingüe (inglés → español)
TRADUCCIONES = {
    'happy': 'feliz', 'sad': 'triste', 'love': 'amor', 'hate': 'odio',
    'good': 'bueno', 'bad': 'malo', 'great': 'genial', 'terrible': 'terrible',
    'work': 'trabajo', 'friend': 'amigo', 'family': 'familia',
    'day': 'día', 'night': 'noche', 'time': 'tiempo', 'morning': 'mañana',
    'hope': 'esperanza', 'wish': 'deseo', 'life': 'vida', 'people': 'gente',
    'best': 'mejor', 'fun': 'divertido', 'awesome': 'increíble',
    'beautiful': 'hermoso', 'lovely': 'encantador', 'nice': 'agradable',
    'perfect': 'perfecto', 'amazing': 'asombroso', 'wonderful': 'maravilloso'
}

# Palabras disponibles para el juego (ordenadas por frecuencia)
PALABRAS_JUEGO = [
    'happy', 'love', 'good', 'great', 'best', 'hope',
    'sad', 'bad', 'hate', 'terrible',
    'work', 'life', 'time', 'day', 'night',
    'friend', 'family', 'people'
]

In [4]:
def crear_sopa_letras_semantica(palabra_objetivo, num_palabras=8, tamano=14):
    """
    Crea una sopa de letras donde las palabras están relacionadas semánticamente.
    
    Args:
        palabra_objetivo: Palabra central del juego
        num_palabras: Cantidad de palabras a ocultar
        tamano: Tamaño de la grilla (tamano x tamano)
    
    Returns:
        tuple: (matriz_sopa, lista_palabras_colocadas)
    """
    
    # Validar que la palabra existe
    if palabra_objetivo not in model_w2v.wv:
        return None, f"❌ Palabra '{palabra_objetivo}' no encontrada en vocabulario"
    
    # Obtener palabras similares
    similares = model_w2v.wv.most_similar(palabra_objetivo, topn=num_palabras*3)
    
    # Filtrar palabras válidas (longitud 3-10, solo letras)
    palabras_validas = []
    for palabra, similitud in similares:
        if 3 <= len(palabra) <= 10 and palabra.isalpha():
            palabras_validas.append((palabra, similitud))
        if len(palabras_validas) >= num_palabras:
            break
    
    # Crear matriz vacía
    sopa = [['' for _ in range(tamano)] for _ in range(tamano)]
    
    # Direcciones posibles (8 direcciones)
    direcciones = [
        (0, 1),   # Horizontal derecha →
        (0, -1),  # Horizontal izquierda ←
        (1, 0),   # Vertical abajo ↓
        (-1, 0),  # Vertical arriba ↑
        (1, 1),   # Diagonal ↘
        (-1, -1), # Diagonal ↖
        (1, -1),  # Diagonal ↙
        (-1, 1)   # Diagonal ↗
    ]
    
    palabras_colocadas = []
    
    # Intentar colocar cada palabra
    for palabra, similitud in palabras_validas:
        palabra_upper = palabra.upper()
        colocada = False
        intentos = 0
        max_intentos = 100
        
        while not colocada and intentos < max_intentos:
            # Posición y dirección aleatorias
            fila = random.randint(0, tamano-1)
            col = random.randint(0, tamano-1)
            direccion = random.choice(direcciones)
            
            # Verificar si se puede colocar
            if puede_colocar(sopa, palabra_upper, fila, col, direccion, tamano):
                colocar_palabra(sopa, palabra_upper, fila, col, direccion)
                
                palabras_colocadas.append({
                    'palabra': palabra_upper,
                    'palabra_es': TRADUCCIONES.get(palabra, palabra),
                    'similitud': float(similitud),
                    'puntos': int(similitud * 100),
                    'posicion': {'fila': fila, 'col': col, 'direccion': direccion}
                })
                colocada = True
            
            intentos += 1
    
    # Rellenar espacios vacíos con letras aleatorias
    letras_comunes = 'AEIOURSTLNMCDPBFGHVJQKWXYZ'  # Ordenadas por frecuencia en inglés
    for i in range(tamano):
        for j in range(tamano):
            if sopa[i][j] == '':
                # Usar letras más comunes para hacer más difícil
                sopa[i][j] = random.choice(letras_comunes[:15])
    
    return sopa, palabras_colocadas


def puede_colocar(sopa, palabra, fila, col, direccion, tamano):
    """Verifica si una palabra cabe en la posición y dirección dadas"""
    df, dc = direccion
    
    for i, letra in enumerate(palabra):
        nueva_fila = fila + i * df
        nueva_col = col + i * dc
        
        # Verificar límites
        if not (0 <= nueva_fila < tamano and 0 <= nueva_col < tamano):
            return False
        
        # Verificar colisiones (puede sobrescribir si es la misma letra)
        celda_actual = sopa[nueva_fila][nueva_col]
        if celda_actual != '' and celda_actual != letra:
            return False
    
    return True


def colocar_palabra(sopa, palabra, fila, col, direccion):
    """Coloca una palabra en la sopa de letras"""
    df, dc = direccion
    
    for i, letra in enumerate(palabra):
        sopa[fila + i * df][col + i * dc] = letra


print("✅ Funciones de generación de sopa de letras cargadas")

✅ Funciones de generación de sopa de letras cargadas


In [5]:
def generar_html_sopa(sopa, palabra_objetivo, palabras_colocadas):
    """
    Genera HTML interactivo ultra-moderno para la sopa de letras.
    
    Características:
    - Diseño glassmorphism con gradientes
    - Animaciones suaves
    - Sistema de selección intuitivo (click y drag)
    - Feedback visual en tiempo real
    - Responsive design
    - Logo de UP integrado
    """
    
    palabra_es = TRADUCCIONES.get(palabra_objetivo, palabra_objetivo)
    tamano = len(sopa)
    
    # Ordenar palabras por similitud (mayor a menor)
    palabras_ordenadas = sorted(palabras_colocadas, key=lambda x: x['similitud'], reverse=True)
    
    # Generar lista de palabras con badges de dificultad
    lista_palabras_html = ""
    for idx, p in enumerate(palabras_ordenadas, 1):
        # Determinar badge de dificultad
        if p['similitud'] >= 0.7:
            badge = '<span class="badge badge-facil">FÁCIL</span>'
        elif p['similitud'] >= 0.5:
            badge = '<span class="badge badge-medio">MEDIO</span>'
        else:
            badge = '<span class="badge badge-dificil">DIFÍCIL</span>'
        
        lista_palabras_html += f"""
        <div class="palabra-item" id="item-{p['palabra']}" data-palabra="{p['palabra']}">
            <div class="palabra-header">
                <span class="palabra-numero">#{idx}</span>
                <span class="palabra-texto">{p['palabra_es'].upper()}</span>
                {badge}
            </div>
            <div class="palabra-info">
                <span class="palabra-ingles">({p['palabra']})</span>
                <span class="palabra-puntos">+{p['puntos']} pts</span>
            </div>
            <div class="similitud-bar">
                <div class="similitud-fill" style="width: {p['similitud']*100}%"></div>
            </div>
        </div>
        """
    
    # Generar grilla de letras
    grilla_html = '<div class="sopa-grilla">'
    for i, fila in enumerate(sopa):
        for j, letra in enumerate(fila):
            grilla_html += f'<div class="celda" data-fila="{i}" data-col="{j}" onclick="toggleCelda(this)">{letra}</div>'
    grilla_html += '</div>'
    
    # Serializar datos para JavaScript
    palabras_json = json.dumps(palabras_colocadas)
    
    html = f"""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sopa de Letras Semántica - Word2Vec</title>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap" rel="stylesheet">
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        :root {{
            --color-primary: #667eea;
            --color-secondary: #764ba2;
            --color-success: #10b981;
            --color-warning: #f59e0b;
            --color-danger: #ef4444;
            --color-info: #3b82f6;
            --shadow-sm: 0 2px 4px rgba(0,0,0,0.1);
            --shadow-md: 0 4px 12px rgba(0,0,0,0.15);
            --shadow-lg: 0 10px 40px rgba(0,0,0,0.2);
            --radius: 16px;
        }}
        
        body {{
            font-family: 'Inter', -apple-system, BlinkMacSystemFont, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
            overflow-x: hidden;
        }}
        
        /* Glassmorphism container */
        .container {{
            max-width: 1400px;
            margin: 0 auto;
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(10px);
            border-radius: var(--radius);
            box-shadow: var(--shadow-lg);
            overflow: hidden;
            animation: fadeIn 0.6s ease-out;
        }}
        
        @keyframes fadeIn {{
            from {{ opacity: 0; transform: translateY(20px); }}
            to {{ opacity: 1; transform: translateY(0); }}
        }}
        
        /* Header con logo */
        .header {{
            background: linear-gradient(135deg, var(--color-primary) 0%, var(--color-secondary) 100%);
            padding: 32px 40px;
            color: white;
            position: relative;
            overflow: hidden;
        }}
        
        .header::before {{
            content: '';
            position: absolute;
            top: -50%;
            right: -10%;
            width: 400px;
            height: 400px;
            background: rgba(255, 255, 255, 0.1);
            border-radius: 50%;
            animation: float 6s ease-in-out infinite;
        }}
        
        @keyframes float {{
            0%, 100% {{ transform: translateY(0) rotate(0deg); }}
            50% {{ transform: translateY(-20px) rotate(180deg); }}
        }}        
        
        @keyframes pulse {{
            0%, 100% {{ opacity: 1; transform: scale(1); }}
            50% {{ opacity: 0.9; transform: scale(1.05); }}
        }}
        
        .header h1 {{
            font-size: 42px;
            font-weight: 800;
            margin-bottom: 8px;
            position: relative;
            z-index: 1;
            text-shadow: 0 2px 10px rgba(0,0,0,0.2);
        }}
        
        .header p {{
            font-size: 16px;
            opacity: 0.95;
            position: relative;
            z-index: 1;
        }}
        
        .palabra-objetivo {{
            background: rgba(255, 255, 255, 0.2);
            padding: 16px 24px;
            border-radius: 12px;
            margin-top: 20px;
            text-align: center;
            font-size: 20px;
            font-weight: 600;
            position: relative;
            z-index: 1;
            backdrop-filter: blur(10px);
        }}
        
        .palabra-objetivo strong {{
            font-size: 28px;
            text-transform: uppercase;
            letter-spacing: 2px;
        }}
        
        /* Stats panel */
        .stats-panel {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 20px;
            padding: 30px 40px;
            background: linear-gradient(to bottom, #f9fafb, #ffffff);
            border-bottom: 1px solid #e5e7eb;
        }}
        
        .stat-card {{
            background: white;
            border-radius: 12px;
            padding: 20px;
            box-shadow: var(--shadow-sm);
            border: 2px solid #e5e7eb;
            transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
        }}
        
        .stat-card:hover {{
            transform: translateY(-4px);
            box-shadow: var(--shadow-md);
            border-color: var(--color-primary);
        }}
        
        .stat-value {{
            font-size: 48px;
            font-weight: 800;
            background: linear-gradient(135deg, var(--color-primary), var(--color-secondary));
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
            line-height: 1;
            margin-bottom: 8px;
        }}
        
        .stat-label {{
            font-size: 14px;
            color: #6b7280;
            font-weight: 600;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}
        
        /* Main content layout */
        .main-content {{
            display: grid;
            grid-template-columns: 1fr 400px;
            gap: 30px;
            padding: 40px;
        }}
        
        /* Grilla de sopa */
        .sopa-grilla {{
            display: grid;
            grid-template-columns: repeat({tamano}, 1fr);
            gap: 4px;
            background: #f3f4f6;
            padding: 20px;
            border-radius: 16px;
            box-shadow: inset 0 2px 8px rgba(0,0,0,0.1);
            max-width: 700px;
            margin: 0 auto;
        }}
        
        .celda {{
            aspect-ratio: 1;
            display: flex;
            align-items: center;
            justify-content: center;
            background: white;
            font-size: 20px;
            font-weight: 700;
            color: #1f2937;
            border-radius: 8px;
            cursor: pointer;
            transition: all 0.2s cubic-bezier(0.4, 0, 0.2, 1);
            user-select: none;
            box-shadow: 0 1px 3px rgba(0,0,0,0.1);
        }}
        
        .celda:hover:not(.encontrada) {{
            background: #e0e7ff;
            transform: scale(1.1);
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.3);
        }}
        
        .celda.seleccionada {{
            background: linear-gradient(135deg, var(--color-primary), var(--color-secondary));
            color: white;
            transform: scale(1.05);
            box-shadow: 0 6px 20px rgba(102, 126, 234, 0.4);
        }}
        
        .celda.encontrada {{
            background: linear-gradient(135deg, #10b981, #059669);
            color: white;
            animation: encontrada 0.6s cubic-bezier(0.68, -0.55, 0.265, 1.55);
            pointer-events: none;
        }}
        
        @keyframes encontrada {{
            0% {{ transform: scale(1); }}
            50% {{ transform: scale(1.3) rotate(10deg); }}
            100% {{ transform: scale(1) rotate(0deg); }}
        }}
        
        /* Sidebar con palabras */
        .sidebar {{
            display: flex;
            flex-direction: column;
            gap: 20px;
        }}
        
        .palabras-container {{
            background: white;
            border-radius: 16px;
            padding: 24px;
            box-shadow: var(--shadow-md);
            max-height: 600px;
            overflow-y: auto;
        }}
        
        .palabras-container h3 {{
            font-size: 20px;
            color: #1f2937;
            margin-bottom: 20px;
            padding-bottom: 12px;
            border-bottom: 3px solid var(--color-primary);
        }}
        
        .palabra-item {{
            background: #f9fafb;
            border-radius: 12px;
            padding: 16px;
            margin-bottom: 12px;
            border: 2px solid #e5e7eb;
            transition: all 0.3s ease;
            cursor: default;
        }}
        
        .palabra-item:hover {{
            border-color: var(--color-primary);
            transform: translateX(4px);
        }}
        
        .palabra-item.encontrada {{
            background: linear-gradient(135deg, #d1fae5, #a7f3d0);
            border-color: var(--color-success);
            opacity: 0.7;
        }}
        
        .palabra-item.encontrada .palabra-texto {{
            text-decoration: line-through;
        }}
        
        .palabra-header {{
            display: flex;
            align-items: center;
            gap: 12px;
            margin-bottom: 8px;
        }}
        
        .palabra-numero {{
            background: linear-gradient(135deg, var(--color-primary), var(--color-secondary));
            color: white;
            width: 32px;
            height: 32px;
            border-radius: 8px;
            display: flex;
            align-items: center;
            justify-content: center;
            font-weight: 700;
            font-size: 14px;
        }}
        
        .palabra-texto {{
            font-size: 18px;
            font-weight: 700;
            color: #1f2937;
            flex: 1;
        }}
        
        .badge {{
            padding: 4px 12px;
            border-radius: 20px;
            font-size: 11px;
            font-weight: 700;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}
        
        .badge-facil {{
            background: #d1fae5;
            color: #065f46;
        }}
        
        .badge-medio {{
            background: #fef3c7;
            color: #92400e;
        }}
        
        .badge-dificil {{
            background: #fee2e2;
            color: #991b1b;
        }}
        
        .palabra-info {{
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 8px;
        }}
        
        .palabra-ingles {{
            font-size: 13px;
            color: #6b7280;
            font-style: italic;
        }}
        
        .palabra-puntos {{
            font-size: 14px;
            font-weight: 700;
            color: var(--color-primary);
        }}
        
        .similitud-bar {{
            width: 100%;
            height: 6px;
            background: #e5e7eb;
            border-radius: 3px;
            overflow: hidden;
        }}
        
        .similitud-fill {{
            height: 100%;
            background: linear-gradient(90deg, var(--color-primary), var(--color-secondary));
            transition: width 0.3s ease;
        }}
        
        /* Botones de acción */
        .acciones {{
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 12px;
        }}
        
        .btn {{
            padding: 16px 24px;
            border: none;
            border-radius: 12px;
            font-size: 16px;
            font-weight: 700;
            cursor: pointer;
            transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
            text-align: center;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}
        
        .btn:hover {{
            transform: translateY(-2px);
            box-shadow: 0 10px 25px rgba(0,0,0,0.15);
        }}
        
        .btn:active {{
            transform: translateY(0);
        }}
        
        .btn-verificar {{
            background: linear-gradient(135deg, var(--color-success), #059669);
            color: white;
            grid-column: 1 / -1;
        }}
        
        .btn-limpiar {{
            background: white;
            color: #6b7280;
            border: 2px solid #e5e7eb;
        }}
        
        .btn-nueva {{
            background: linear-gradient(135deg, var(--color-primary), var(--color-secondary));
            color: white;
        }}
        
        /* Animación de victoria */
        .confetti {{
            position: fixed;
            width: 10px;
            height: 10px;
            pointer-events: none;
            z-index: 9999;
        }}
        
        @keyframes confetti-fall {{
            to {{
                transform: translateY(100vh) rotate(360deg);
                opacity: 0;
            }}
        }}
        
        /* Modal de victoria */
        .modal {{
            display: none;
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: rgba(0, 0, 0, 0.8);
            backdrop-filter: blur(10px);
            z-index: 10000;
            align-items: center;
            justify-content: center;
        }}
        
        .modal-content {{
            background: white;
            padding: 60px;
            border-radius: 24px;
            text-align: center;
            max-width: 500px;
            animation: modalSlideIn 0.5s cubic-bezier(0.68, -0.55, 0.265, 1.55);
        }}
        
        @keyframes modalSlideIn {{
            from {{
                opacity: 0;
                transform: scale(0.5) translateY(-100px);
            }}
            to {{
                opacity: 1;
                transform: scale(1) translateY(0);
            }}
        }}
        
        .modal h2 {{
            font-size: 48px;
            margin-bottom: 20px;
            background: linear-gradient(135deg, var(--color-primary), var(--color-secondary));
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
        }}
        
        .modal-emoji {{
            font-size: 80px;
            margin-bottom: 20px;
            animation: bounce 1s infinite;
        }}
        
        @keyframes bounce {{
            0%, 100% {{ transform: translateY(0); }}
            50% {{ transform: translateY(-20px); }}
        }}
        
        /* Scrollbar personalizado */
        ::-webkit-scrollbar {{
            width: 8px;
        }}
        
        ::-webkit-scrollbar-track {{
            background: #f3f4f6;
            border-radius: 4px;
        }}
        
        ::-webkit-scrollbar-thumb {{
            background: linear-gradient(135deg, var(--color-primary), var(--color-secondary));
            border-radius: 4px;
        }}
        
        ::-webkit-scrollbar-thumb:hover {{
            background: linear-gradient(135deg, var(--color-secondary), var(--color-primary));
        }}
        
        /* Responsive */
        @media (max-width: 1200px) {{
            .main-content {{
                grid-template-columns: 1fr;
            }}
            
            .palabras-container {{
                max-height: 400px;
            }}
        }}
        
        @media (max-width: 768px) {{
            .sopa-grilla {{
                gap: 2px;
                padding: 10px;
            }}
            
            .celda {{
                font-size: 16px;
            }}
            
            .header h1 {{
                font-size: 28px;
            }}
            
            .stats-panel {{
                grid-template-columns: 1fr;
            }}
        }}
    </style>
</head>
<body>
    <div class="container">
        <!-- Header -->
        <div class="header">
            </div>
            <div class="palabra-objetivo">
                🎯 Encuentra palabras relacionadas con: <strong>{palabra_es.upper()}</strong> <span style="opacity: 0.7;">({palabra_objetivo.upper()})</span>
            </div>
        </div>
        
        <!-- Stats Panel -->
        <div class="stats-panel">
            <div class="stat-card">
                <div class="stat-value" id="puntos">0</div>
                <div class="stat-label">💰 Puntos</div>
            </div>
            <div class="stat-card">
                <div class="stat-value" id="encontradas">0</div>
                <div class="stat-label">✓ Encontradas</div>
            </div>
            <div class="stat-card">
                <div class="stat-value" id="restantes">{len(palabras_colocadas)}</div>
                <div class="stat-label">⏳ Restantes</div>
            </div>
        </div>
        
        <!-- Main Content -->
        <div class="main-content">
            <!-- Grilla de letras -->
            <div>
                {grilla_html}
            </div>
            
            <!-- Sidebar -->
            <div class="sidebar">
                <div class="palabras-container">
                    <h3>📝 Palabras a Encontrar</h3>
                    <div id="palabras-lista">
                        {lista_palabras_html}
                    </div>
                </div>
                
                <div class="acciones">
                    <button class="btn btn-verificar" onclick="verificarSeleccion()">✓ Verificar</button>
                    <button class="btn btn-limpiar" onclick="limpiarSeleccion()">✗ Limpiar</button>
                    <button class="btn btn-nueva" onclick="location.reload()">🔄 Nueva Palabra</button>
                </div>
            </div>
        </div>
    </div>
    
    <!-- Modal de Victoria -->
    <div class="modal" id="modalVictoria">
        <div class="modal-content">
            <div class="modal-emoji">🎉</div>
            <h2>¡FELICITACIONES!</h2>
            <p style="font-size: 24px; margin: 20px 0;">Completaste la sopa de letras</p>
            <p style="font-size: 18px; color: #6b7280;">Puntos totales: <strong id="puntos-final" style="color: var(--color-primary);">0</strong></p>
            <button class="btn btn-nueva" onclick="location.reload()" style="margin-top: 30px;">🔄 Jugar de Nuevo</button>
        </div>
    </div>
    
    <script>
        // Datos del juego
        const PALABRAS = {palabras_json};
        const TAMANO_GRILLA = {tamano};
        
        // Estado del juego
        let celdasSeleccionadas = [];
        let puntosTotal = 0;
        let palabrasEncontradas = new Set();
        let seleccionando = false;
        
        // Inicializar evento de drag
        document.addEventListener('mousedown', (e) => {{
            if (e.target.classList.contains('celda') && !e.target.classList.contains('encontrada')) {{
                seleccionando = true;
                toggleCelda(e.target);
            }}
        }});
        
        document.addEventListener('mouseover', (e) => {{
            if (seleccionando && e.target.classList.contains('celda') && !e.target.classList.contains('encontrada')) {{
                if (!e.target.classList.contains('seleccionada')) {{
                    toggleCelda(e.target);
                }}
            }}
        }});
        
        document.addEventListener('mouseup', () => {{
            seleccionando = false;
        }});
        
        function toggleCelda(celda) {{
            if (celda.classList.contains('encontrada')) return;
            
            if (celda.classList.contains('seleccionada')) {{
                celda.classList.remove('seleccionada');
                celdasSeleccionadas = celdasSeleccionadas.filter(c => c !== celda);
            }} else {{
                celda.classList.add('seleccionada');
                celdasSeleccionadas.push(celda);
            }}
        }}
        
        function verificarSeleccion() {{
            if (celdasSeleccionadas.length === 0) {{
                mostrarMensaje('⚠️ Selecciona al menos una letra', 'warning');
                return;
            }}
            
            // Obtener palabra formada
            let palabraFormada = celdasSeleccionadas.map(c => c.textContent).join('');
            let palabraReversa = palabraFormada.split('').reverse().join('');
            
            // Buscar en ambas direcciones
            let encontrada = PALABRAS.find(p => 
                (p.palabra === palabraFormada || p.palabra === palabraReversa) &&
                !palabrasEncontradas.has(p.palabra)
            );
            
            if (encontrada) {{
                // ¡Palabra encontrada!
                marcarPalabraEncontrada(encontrada);
            }} else {{
                // Palabra incorrecta
                mostrarError();
            }}
        }}
        
        function marcarPalabraEncontrada(palabra) {{
            // Marcar celdas
            celdasSeleccionadas.forEach(celda => {{
                celda.classList.remove('seleccionada');
                celda.classList.add('encontrada');
            }});
            
            // Actualizar estadísticas
            palabrasEncontradas.add(palabra.palabra);
            puntosTotal += palabra.puntos;
            
            document.getElementById('puntos').textContent = puntosTotal;
            document.getElementById('encontradas').textContent = palabrasEncontradas.size;
            document.getElementById('restantes').textContent = PALABRAS.length - palabrasEncontradas.size;
            
            // Marcar en la lista
            let item = document.querySelector(`[data-palabra="${{palabra.palabra}}"]`);
            if (item) {{
                item.classList.add('encontrada');
            }}
            
            // Limpiar selección
            celdasSeleccionadas = [];
            
            // Verificar victoria
            if (palabrasEncontradas.size === PALABRAS.length) {{
                setTimeout(() => {{
                    mostrarVictoria();
                }}, 500);
            }}
        }}
        
        function mostrarError() {{
            celdasSeleccionadas.forEach(celda => {{
                celda.style.background = '#ef4444';
                celda.style.color = 'white';
                setTimeout(() => {{
                    celda.style.background = '';
                    celda.style.color = '';
                    celda.classList.remove('seleccionada');
                }}, 400);
            }});
            celdasSeleccionadas = [];
        }}
        
        function limpiarSeleccion() {{
            celdasSeleccionadas.forEach(c => c.classList.remove('seleccionada'));
            celdasSeleccionadas = [];
        }}
        
        function mostrarVictoria() {{
            // Confetti
            for (let i = 0; i < 100; i++) {{
                setTimeout(() => {{
                    crearConfetti();
                }}, i * 30);
            }}
            
            // Modal
            document.getElementById('puntos-final').textContent = puntosTotal;
            document.getElementById('modalVictoria').style.display = 'flex';
        }}
        
        function crearConfetti() {{
            const confetti = document.createElement('div');
            confetti.className = 'confetti';
            confetti.style.left = Math.random() * 100 + '%';
            confetti.style.top = '-10px';
            confetti.style.background = `hsl(${{Math.random() * 360}}, 70%, 60%)`;
            confetti.style.animation = `confetti-fall ${{2 + Math.random() * 2}}s linear forwards`;
            document.body.appendChild(confetti);
            
            setTimeout(() => confetti.remove(), 4000);
        }}
        
        function mostrarMensaje(texto, tipo) {{
            console.log(`[${{tipo}}] ${{texto}}`);
        }}
        
        // Prevenir selección de texto
        document.addEventListener('selectstart', (e) => {{
            if (e.target.classList.contains('celda')) {{
                e.preventDefault();
            }}
        }});
    </script>
</body>
</html>
    """
    
    return html

print("✅ Función de generación HTML cargada")

✅ Función de generación HTML cargada


## 🎮 Ejecutar el Juego

Selecciona una palabra objetivo y genera la sopa de letras:

In [6]:
# Configuración del juego
PALABRA_OBJETIVO = 'happy'  # Cambiar por: 'love', 'sad', 'work', 'life', etc.
NUM_PALABRAS = 8            # Cantidad de palabras a ocultar (recomendado: 6-10)
TAMANO_GRILLA = 14          # Tamaño de la grilla (recomendado: 12-16)

print(f"🎯 Generando sopa de letras para: '{PALABRA_OBJETIVO}'...\n")

# Generar sopa de letras
sopa, palabras = crear_sopa_letras_semantica(
    palabra_objetivo=PALABRA_OBJETIVO,
    num_palabras=NUM_PALABRAS,
    tamano=TAMANO_GRILLA
)

if sopa is None:
    print(f"❌ Error: {palabras}")
else:
    print(f"✅ Sopa de letras generada exitosamente")
    print(f"   • Tamaño: {len(sopa)}x{len(sopa)}")
    print(f"   • Palabras ocultas: {len(palabras)}\n")
    
    print("📊 Palabras y similitudes:")
    for i, p in enumerate(sorted(palabras, key=lambda x: x['similitud'], reverse=True), 1):
        print(f"   {i}. {p['palabra_es'].upper():12} ({p['palabra']:10}) - {p['similitud']:.2%} - {p['puntos']} pts")
    
    print("\n🎮 Cargando juego interactivo...")
    
    # Generar y mostrar HTML
    html_juego = generar_html_sopa(sopa, PALABRA_OBJETIVO, palabras)
    display(HTML(html_juego))

🎯 Generando sopa de letras para: 'happy'...

✅ Sopa de letras generada exitosamente
   • Tamaño: 14x14
   • Palabras ocultas: 8

📊 Palabras y similitudes:
   1. HAPPYY       (HAPPYY    ) - 65.84% - 65 pts
   2. THRILLED     (THRILLED  ) - 62.07% - 62 pts
   3. PLEASED      (PLEASED   ) - 60.05% - 60 pts
   4. TRISTE       (SAD       ) - 59.42% - 59 pts
   5. UPSET        (UPSET     ) - 58.36% - 58 pts
   6. BLESSED      (BLESSED   ) - 57.77% - 57 pts
   7. THANKFUL     (THANKFUL  ) - 57.08% - 57 pts
   8. UNHAPPY      (UNHAPPY   ) - 55.84% - 55 pts

🎮 Cargando juego interactivo...


## 📚 Cómo Funciona

### 1. **Generación de palabras relacionadas**
```python
similares = model_w2v.wv.most_similar('happy', topn=10)
# Retorna: [('love', 0.85), ('good', 0.72), ('great', 0.68), ...]
```

### 2. **Cálculo de puntos**
Los puntos = similitud × 100:
- `similarity('happy', 'love') = 0.85` → **85 puntos**
- `similarity('happy', 'good') = 0.72` → **72 puntos**
- `similarity('happy', 'sad') = 0.31` → **31 puntos**

### 3. **Colocación en la grilla**
Las palabras se colocan en 8 direcciones posibles:
- Horizontal (→ ←)
- Vertical (↓ ↑)
- Diagonal (↘ ↖ ↙ ↗)

### 4. **Validación pedagógica**
Este juego demuestra que Word2Vec capturó **relaciones semánticas reales**:
- Si "LOVE" está cerca de "HAPPY", el modelo aprendió bien
- Si las similitudes no tienen sentido, indica problemas en el entrenamiento

---

## 🎯 Interpretación de Resultados

**Palabras con alta similitud (>70%)**:
- Son conceptos muy relacionados en el corpus
- Aparecieron frecuentemente en contextos similares
- Ejemplo: "happy" y "love" comparten muchos tweets positivos

**Palabras con similitud media (50-70%)**:
- Tienen relación semántica pero menos directa
- Ejemplo: "happy" y "day" (contextos como "happy day")

**Palabras con baja similitud (<50%)**:
- Relación débil o contraste semántico
- Ejemplo: "happy" y "sad" (opuestos pero comparten el dominio emocional)

---

**Autor:** Omar Alejandro  
**Diplomatura en IA - Universidad de Palermo**  
**Trabajo Práctico 3: Análisis de Sentimientos con NLP**
